<a href="https://colab.research.google.com/github/kv3n/shape-analysis/blob/master/shape_preserving_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-graphics
!pip install openmesh

     |████████████████████████████████| 8.9MB 4.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/2e/8a/f1/95efd915e63be94f3ed3cd5ffe3accb71c5654e9a85b9804ac
Successfully built openmesh


In [0]:
from tensorflow_graphics.math import vector
from tensorflow_graphics.notebooks import threejs_visualization
from tensorflow_graphics.notebooks import mesh_viewer
import openmesh as om
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [57]:
mesh = om.read_trimesh('/content/gdrive/My Drive/SELF/PROJECT/shape-analysis/shapes/curiosity/model.obj')

# Load Vertices
vertices = []
for point in mesh.points():
  vertices.append(point)
vertices = np.array(vertices)

# Load Triangles
faces = []
for fh in mesh.faces():
  face = []
  for vh in mesh.fv(fh):
    face.append(vh.idx())
  faces.append(face)
faces = np.array(faces)

mesh_data = {'vertices': vertices[:, ...], 'faces': faces[:, ...]}

False


In [55]:
mesh_viewer.Viewer(mesh_data)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>